## Q1. What is the difference between __getattr__ and __getattribute__?


Ans: __getattribute__ is used to find an attribute of a class. It raises an AttributeError of it fails to find an attribute of a class. __getattr__ is implemented latter if AttributeError is generated by __getattribute__, but for this __getattribute__ and __getattr__ both has to be defined in same class. If no attribute is found, __getattr__ returns a default value. So key difference is that __getattr__ is called for attributes that don't actually exist on a class.